In [1]:
#################################################################
# File       : Read_and_Display_OMETIFF_CZI.ipynb
# Version    : 0.3
# Author     : czsrh
# Date       : 02.11.2019
# Insitution : Carl Zeiss Microscopy GmbH
#
# Disclaimer: Just for testing - Use at your own risk.
# Feedback or Improvements are welcome.
##################################################################

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from apeer_ometiff_library import io, processing, omexmlClass
import os
import numpy as np
import ipywidgets as widgets
import napari
import imgfileutils as imf

In [3]:
# define your testfiles here

imgdict = {
    1:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=10_Z=15_CH=2_DCV_small.czi',
    2:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=10_Z=15_CH=2_DCV_small.ome.tiff',
    3:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=10_Z=15_CH=2_DCV_small_Fiji.ome.tiff',
    4:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=15_Z=20_CH=2_DCV.czi',
    5:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\NeuroSpheres_DCV_A635_A488_A405.czi',
    6:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CZI_DimorderTZC.czi'
}

filename = imgdict[3]

In [4]:

def read_ometiff(input_path):
    
    from skimage.external import tifffile
    
    with tifffile.TiffFile(input_path) as tif:
        array = tif.asarray()
        print(array.shape)
        omexml_string = tif[0].image_description.decode("utf-8")

    # Turn Ome XML String to an Bioformats object for parsing
    metadata = omexmlClass.OMEXML(omexml_string)

    print(array.shape)
    
    # Parse pixel sizes
    pixels = metadata.image(0).Pixels
    size_c = pixels.SizeC
    size_t = pixels.SizeT
    size_z = pixels.SizeZ
    size_x = pixels.SizeX
    size_y = pixels.SizeY

    # Expand image array to 5D of order (T, Z, C, X, Y)
    if size_c == 1:
        array = np.expand_dims(array, axis=-3)
        print('C=1', array.shape)
    if size_z == 1:
        array = np.expand_dims(array, axis=-4)
        print('Z=1', array.shape)
    if size_t == 1:
        array = np.expand_dims(array, axis=-5)
        print('T=1', array.shape)

    return array, omexml_string

(array, omexml) = read_ometiff(filename)



(10, 15, 2, 256, 256)
(10, 15, 2, 256, 256)


In [5]:
# read metadata and array differently for OME-TIFF or CZI data
if filename.lower().endswith('.ome.tiff') or filename.lower().endswith('.ome.tif'):
    
    # Return value is an array of order (T, Z, C, X, Y)
    (array, omexml) = io.read_ometiff(filename)
    metadata = imf.get_metadata(filename, series=0)
    print('Array Shape: ', array.shape)
    print('Sizes BF: ', metadata['Sizes BF'])
    print(metadata['DimOrder BF'])
    print(metadata['DimOrder BF Array'])
    
if filename.lower().endswith('.czi'):

    # get the array and the metadata
    array, metadata = imf.get_array_czi(filename, replacezero=False)
    print('Array Shape: ', array.shape)
    print(metadata['Shape'])
    print(metadata['Axes'])

Image Type:  ometiff
Getting OME-TIFF Metadata ...
Array Shape:  (10, 15, 2, 256, 256)
Sizes BF:  [1, 10, 15, 2, 256, 256]
XYCZT
TZCYX


In [6]:
# display data using ipy widgets
if metadata['Extension'] == 'ome.tiff':
    ui, out = imf.create_ipyviewer_ome_tiff(array, metadata)
if metadata['Extension'] == 'czi':
    ui, out = imf.create_ipyviewer_czi(array, metadata)

display(ui, out)

Output()

In [ ]:
import time

# switch to qt5 backend for napari viewer and wait a few seconds
%gui qt5
time.sleep(5)

In [ ]:
# try to configure napari automatiaclly based on metadata
imf.show_napari(array, metadata)

In [ ]:
# configure napari viewer manually - check array shape and dimensions order carefully 
    
# get the scalefactors
scalefactors = imf.get_scalefactor(metadata)
print(scalefactors)

array = np.squeeze(array, axis=(0, 1))

viewer = napari.Viewer()
# add every channel as a single layer
for ch in range(metadata['SizeC']):
    chname = metadata['Channels'][ch]
    viewer.add_image(array[ch, :, :, :], name=chname, scale=(1, scalefactors['zx'], 1, 1))